In [41]:
#import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from diffprivlib import tools as dp
from diffprivlib.mechanisms import Binary
from diffprivlib.mechanisms import Geometric
import os
from collections import Counter

%store -r races
%store -r real_order

In [42]:
###########################################################################################################################
#All this code does is see approximately how many unique entries for AGE, SEX, and RACE exist in the dataset 
#(does slightly overcount, this is just for reference)
###########################################################################################################################

def create_block_data(path):
    df = pd.read_csv(path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.loc[:, ~df.columns.str.contains('sex')]
    df = df.loc[:, ~df.columns.str.contains('household_tenure')]
    df = df.loc[:, ~df.columns.str.contains('id')]
    print(df)
    return df

In [43]:
def pick_swapvars(df):
    #all v is collection of all row values
    all_v = []

    #add all rows to list all_v
    for i, row in df.iterrows():
        all_v.append([row['age'], row['race'], row['household_size'], row['hispanic']])
        

    #drop duplicates based upon age, sex, and race from df
    df_drop = df.drop_duplicates(subset = ['age', 'race', 'household_size', 'hispanic'])
        
    #the list of ids of all rows that contain unique entries
    swap = []
    
    #for each item in uniques, check how many times it occurs in original data
    for i, row in df_drop.iterrows():
        #list containing age, sex, race
        check = [row['age'], row['race'], row['household_size'], row['hispanic']]
        dcount = 0 
        #for all rows in the dataframe
        for a in all_v:
            #check if the age, sex, and races match the row
            if a==check:
                #if they do match, incremement by 1
                dcount +=1
        #if only present once, add to swaps
        if dcount == 1:
            swap.append('ah')
            
               
    return len(swap)

In [48]:
def import_countydata(county):
    #import csv file as dataframe
    df = pd.read_csv (r'../homemade_data/'+county+'.csv')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    #transfer race representation to include hispanic data:
    #if not hispanic: index is 0-62
    #if hispanic: index is 63-126
        #so, if hispanic, add 63 to the existing race value
    for index, row in df.iterrows():
        d = row['race']
        h = row['hispanic']
        if d in races and h==0:
            df.at[index, 'race'] = real_order.index(races.index(d))
        elif d in races and h==1:
            df.at[index, 'race'] = real_order.index((races.index(d))+63)

    return df

In [53]:
def pick_swapvars(df):
    #all v is collection of all row values
    all_v = []

    #add all rows to list all_v
    for i, row in df.iterrows():
        all_v.append([[row['age'], row['sex'], row['race']], row['id']])
        
    #drop duplicates based upon age, sex, and race from df
    df_drop = df.drop_duplicates(subset = ['age', 'sex', 'race'])
        
    #the list of ids of all rows that contain unique entries
    swap = []
    
    #for each item in uniques, check how many times it occurs in original data
    for i, row in df_drop.iterrows():
        #list containing age, sex, race
        check = [row['age'], row['sex'], row['race']]
        dcount = 0 
        #for all rows in the dataframe
        for a in all_v:
            #check if the age, sex, and races match the row
            if a[0]==check:
                #if they do match, incremement by 1
                dcount +=1
        #if only present once, add to swaps
        if dcount == 1:
            swap.append(row['id']-1)
            
               
    return len(swap)

In [55]:
counties = ['alameda']
for county in counties:    
    #df_first = create_block_data('../homemade_data/'+county+'.csv')
    df_first = import_countydata(county)
    df_orig = df_first.copy()
#     for index, row in df_orig.iterrows():
#         d = row['race']
#         df_orig.at[index, 'race'] = races.index(d)
        #comment out for not 15 bucket age
#         a = row['age']
#         if a<15:
#             df_orig.at[index, 'age'] = 0
#         elif a<30:
#             df_orig.at[index, 'age'] = 1
#         elif a<45:
#             df_orig.at[index, 'age'] = 2
#         elif a<60:
#             df_orig.at[index, 'age'] = 3
#         elif a<75:
#             df_orig.at[index, 'age'] = 4
#         else:
#             df_orig.at[index, 'age'] = 5
        
    orig = df_orig.shape[0]
    new = pick_swapvars(df_orig)
    print(county, new/orig, orig, new)

alameda 0.2330300909727082 1429 333
